In [1]:
###############################################################################
######################   UsedCarPrices_CarGurus  ##############################
######################  Regression - Nonlinear   ##############################
######################        MLP Methods        ##############################
###############################################################################

In [2]:
# Install and import packages
!pip install keras-tuner
from tensorflow import keras
import keras_tuner
from keras_tuner import BayesianOptimization
import os
import random
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib as plt
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns

print('\n')
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

     |████████████████████████████████| 133 kB 4.2 MB/s 


TensorFlow version: 2.8.0
Eager execution is: True
Keras version: 2.8.0
Num GPUs Available:  1


In [ ]:
# Reproducibility in TF and Pytorch 
def init_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
    os.environ['TF_CUDNN_DETERMINISTIC'] ='true'
    os.environ['TF_DETERMINISTIC_OPS'] = 'true'
    tf.random.set_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                                config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    return sess

In [ ]:
init_seeds(seed=42)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/UsedCarsCarGurus/Data/

/content/drive/MyDrive/UsedCarsCarGurus/Data


In [ ]:
df = pd.read_csv('usedCars_final.csv', low_memory=False)
print('Number of rows and columns:', df.shape)

Number of rows and columns: (327912, 28)


In [ ]:
%cd /content/drive/MyDrive/UsedCarsCarGurus/ML/MLP/Results/HPO/

In [ ]:
# Prepare for partitioning data
X = df.drop(['price'],axis=1)
y = df['price']

In [ ]:
# Set up train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 42)

In [ ]:
# Train: Create dummy variables for categorical variables
X_train = pd.get_dummies(X_train, drop_first=True)

# Test: Create dummy variables for categorical variables
X_test = pd.get_dummies(X_test, drop_first=True)

In [ ]:
# MinMax Scaling
mn = MinMaxScaler()
X_train = pd.DataFrame(mn.fit_transform(X_train))
X_test = pd.DataFrame(mn.fit_transform(X_test))

In [ ]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

%load_ext tensorboard

In [ ]:
# Set up callbacks
filepath = 'MLP_weights_only_b1_HPO6.h5'

checkpoint_dir = os.path.dirname(filepath)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_folder,
                                                      histogram_freq=1)

callbacks_list = [EarlyStopping(monitor='val_loss', patience = 5),
                  ModelCheckpoint(filepath, monitor='mse', 
                                  save_best_only = True, mode='min'), 
                  tensorboard_callback]


In [ ]:
# Define model for HPO
def build_model(hp): 
  model = keras.Sequential()
  for i in range(hp.Int("num_layers", 20, 30)):
        model.add(tf.keras.layers.Dense( units=hp.Int("layer_size" + str(i), min_value=30, max_value=80, step=5),
                activation="relu", kernel_initializer='normal'))
  model.add(tf.keras.layers.Dropout(0.3))
  model.add(Dense(1))
  model.compile(loss="mae", metrics=["mse"], optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-4, 1e-5, 1e-6])))
  return model

In [ ]:
# Define the search conditions 
tuner = BayesianOptimization(
    build_model,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory="MLP_b1_HPO6",
    project_name="MLP_b1_HPO6"
)

In [ ]:
# Print a summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 30, 'step': 1, 'sampling': None}
layer_size0 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size1 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size2 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size3 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size4 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size5 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size6 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 80, 'step': 5, 'sampling': None}
layer_size7 (I

In [ ]:
# Begin the search for the best hyperparameters 
tuner.search(X_train, y_train, epochs=2, validation_split=0.2, batch_size=1,
             callbacks=callbacks_list)

Trial 18 Complete [01h 07m 31s]
val_loss: 3126.0699462890625

Best val_loss So Far: 3029.4842529296875
Total elapsed time: 19h 59m 11s

Search: Running Trial #19

Hyperparameter    |Value             |Best Value So Far 
num_layers        |30                |28                
layer_size0       |45                |55                
layer_size1       |30                |40                
layer_size2       |80                |80                
layer_size3       |30                |45                
layer_size4       |30                |40                
layer_size5       |30                |30                
layer_size6       |30                |30                
layer_size7       |30                |30                
layer_size8       |30                |30                
layer_size9       |65                |80                
layer_size10      |80                |80                
layer_size11      |50                |45                
layer_size12      |65                |8

In [ ]:
# Retrieve the best model(s)
models = tuner.get_best_models(num_models=3)
models

In [ ]:
# Print a summary of the results from the trials
tuner.results_summary()

In [ ]:
# Fit best model from HPO 